In [ ]:
from helpers import datastore, registry
import mdsisclienttools.auth.TokenManager as ProvenaAuth
from utils import pprint_json

PROVENA_DOMAIN = "dev.rrap-is.com"

# Edit this to point to the Keycloak instance for your Provena instance
kc_endpoint = "https://auth.dev.rrap-is.com/auth/realms/rrap"

stage = "DEV"
registry_endpoint = "https://registry-api.{}".format(PROVENA_DOMAIN)
provenance_endpoint = "https://prov-api.{}".format(PROVENA_DOMAIN)
data_store_endpoint = "https://data-api.{}".format(PROVENA_DOMAIN)
job_endpoint =  "https://job-api.{}".format(PROVENA_DOMAIN)

# sets up auth connections - could potentially open browser window if not signed
# in recently - caches in .tokens.json - ensure this is included in gitignore
provena_auth = ProvenaAuth.DeviceFlowManager(
    stage=stage,
    keycloak_endpoint=kc_endpoint
)

# expose the get auth function which is used for provena methods 
get_auth = provena_auth.get_auth



In [ ]:
dataset_id = "10378.1/1764769"
reason="For demonstration Purposes, creating a second version of the dataset."
version_response = datastore.version_dataset(datastore_endpoint=data_store_endpoint, dataset_id=dataset_id, reason=reason, auth=get_auth())
pprint_json(version_response)


In [ ]:
new_version_id = version_response['new_version_id']
metadata = registry.fetch_dataset_metadata(
    registry_endpoint=registry_endpoint, id=new_version_id, auth=get_auth())

updated_metadata = metadata  # "copy" to edit
updated_metadata['dataset_info']["description"] = "This is a new version of the dataset!"
updated_metadata['dataset_info']['spatial_info']['resolution'] = 0.0013
updated_metadata['associations']["point_of_contact"] = "Peter Baker"
updated_metadata['dataset_info']['temporal_info'] = {
    "duration": {
        "begin_date": "2020-07-24",
        "end_date": "2020-07-25"
    },
    "resolution": "P1Y2M10DT2H30M"
}


update_response = datastore.update_dataset(datastore_endpoint=data_store_endpoint,
                                           updated_metadata=updated_metadata, dataset_id=dataset_id, reason=reason, auth=get_auth())
pprint_json(update_response)